# Lab session 6: Association Analysis

## Introduction

The aim of this lab (Lab session 6) is for students to get experience with **Association Analysis** covered in week 8, by using typical Python libraries.


This session starts with a tutorial that uses examples to introduce you to  practical knowledge. We highly recommend that you read the following tutorials if you need a gentler introduction to the libraries that we use:
- [Mlxtend: Apriori](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/)
- [Numpy quickstart tutorial](https://numpy.org/devdocs/user/quickstart.html)
- [Numpy: basic broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html)
- [Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html)
- [Matplotlib](https://matplotlib.org/tutorials/introductory/pyplot.html)
- [Seaborn](https://seaborn.pydata.org/tutorial/relational.html)
- [Scikit-learn](https://scikit-learn.org/stable/tutorial/basic/tutorial.html)

## 1. Frequent itemsets

In order to present functionalities for association analysis in Python, we adapt an example from the ``mlxtend`` documentation.

Consider a dataset composed of five transactions. This dataset is represented by a list of five elements, each of which is a list of items bought during a trip to a supermarket.

In [1]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

The library ``mlxtend`` requires that each transaction is represented by a binary vector where each element indicates the presence or absence of a specific item.

The method ``TransactionEncoder.fit_transform`` can be used to convert the dataset created above into this expected format. This method returns a binary matrix (numpy array) where each transaction corresponds to a row and each column corresponds to an item.

In [2]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit_transform(dataset)
print(te_ary)

[[False False False  True False  True  True  True  True False  True]
 [False False  True  True False  True False  True  True False  True]
 [ True False False  True False  True  True False False False False]
 [False  True False False False  True  True False False  True  True]
 [False  True False  True  True  True False False  True False False]]


The item corresponding to each column is stored by the ``TransactionEncoder`` object in a variable called ``columns_``. This variable can be used to create a ``DataFrame`` that conveniently represents the transaction dataset.

In [3]:
import pandas as pd

df = pd.DataFrame(te_ary, columns=te.columns_)
display(df)

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


The ``mlxtend`` function ``apriori`` receives a ``DataFrame`` that represents a transaction dataset and a parameter that specifies the support threshold. This function returns a ``DataFrame`` that contains one row for each frequent itemset. Each row contains a python ``frozenset`` that represents the itemset (by column indices) and a number that represents the support of this itemset.

In [4]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(df, min_support=0.6)
display(frequent_itemsets)

itemset = frequent_itemsets.loc[5]
print('Itemset: {0}. Support: {1}.'.format(itemset['itemsets'], itemset['support']))

,support,itemsets
0,0.8,(3)
1,1.0,(5)
2,0.6,(6)
3,0.6,(8)
4,0.6,(10)
5,0.8,"(3, 5)"
6,0.6,"(8, 3)"
7,0.6,"(5, 6)"
8,0.6,"(8, 5)"
9,0.6,"(10, 5)"


Itemset: frozenset({3, 5}). Support: 0.8.


Conveniently, if the parameter ``use_colnames`` is set to ``True``,  the ``mlxtend`` function ``apriori`` may instead return a ``DataFrame`` that represents itemsets by ``frozensets`` of item names.

In [5]:
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
display(frequent_itemsets)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.8,(Eggs)
1,1.0,(Kidney Beans)
2,0.6,(Milk)
3,0.6,(Onion)
4,0.6,(Yogurt)
5,0.8,"(Kidney Beans, Eggs)"
6,0.6,"(Onion, Eggs)"
7,0.6,"(Milk, Kidney Beans)"
8,0.6,"(Onion, Kidney Beans)"
9,0.6,"(Kidney Beans, Yogurt)"


Using typical ``pandas`` functionalities, it is easy to include a column in such a ``DataFrame`` to register the number of items in each frequent itemset, which can be used to filter itemsets by length.

In [6]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x)) # length of each frozenset
print('Frequent 3-itemsets:')
display(frequent_itemsets[frequent_itemsets['length'] == 3])

Frequent 3-itemsets:


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets,length
10,0.6,"(Onion, Kidney Beans, Eggs)",3


It is also easy to create a ``dict`` that maps any frequent itemset (represented by a ``frozenset``) to its support.

In [7]:
support = {}
for _, row in frequent_itemsets.iterrows():
    support[row['itemsets']] = row['support']

itemset = frozenset(['Onion', 'Eggs'])
print('Itemset: {0}. Support: {1}.'.format(itemset, support[itemset]))

Itemset: frozenset({'Onion', 'Eggs'}). Support: 0.6.


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 2. Association rules

The ``mlxtend`` function ``association_rules`` receives a ``DataFrame`` that represents the set of frequent itemsets and returns a ``DataFrame`` that represents strong association rules for a specified confidence threshold. Each row in the resulting  ``DataFrame`` contains an association rule together with some potentially useful measures (we have not covered lift, leverage, or conviction).

In [8]:
from mlxtend.frequent_patterns import association_rules

strong_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
display(strong_rules)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Kidney Beans),(Eggs),1.0,0.8,0.8,0.80,1.00,0.00,1.0,0.0
1,(Eggs),(Kidney Beans),0.8,1.0,0.8,1.00,1.00,0.00,inf,0.0
2,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf,0.5
3,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6,1.0
4,(Milk),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,0.0
5,(Onion),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,0.0
6,(Yogurt),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,0.0
7,"(Onion, Kidney Beans)",(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf,0.5
8,"(Onion, Eggs)",(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,0.0
9,"(Kidney Beans, Eggs)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6,1.0


## Exercise



The sample dataset to be used is one of Kaggle’s e-commerce resources containing historical sales transactions of electronic items. The dataset can be found in QM+.

The tasks to be completed for this exercise are:

1) Import the dataset </br>
2) Keep only the columns: 'Order ID', 'Product' </br>
3) Reset the index of the dataframe </br>
4) Use the following command for structuring the dataframe into a form/format for analysis: </br> pivot_table(index = 'Order ID', columns = 'Product', aggfunc = 'count') </br>
5) Check the resulting dataframe and perform a necessary pre-processing technique </br>
6) Convert the columns to boolean </br>
7) Identify the support frequency of items </br>
The support frequency is the number of times any individual item is sold (here we only count each item once per order) divided by the total number of transactions (or orders in this example). Which are the top-5 most frequent selling items? Can this insight help a retailer also boost the sales of less frequent selling items by pairing them with these high frequency lower cost electronic goods? Which products would make sense to pair together given this insight? </br>
8) Run the apriori algorithm from mlxtend </br>
9) Compute items that sell at minimum in a percentage of all orders placed in the month (e.g. try min_support=0.00000001) </br>
10) Compute pairs of orders that have a meaningful measure (e.g lift of more than 1; confidence of more than 0.1) in their sales when marketed together





In [9]:
import numpy as np

# insert code
sales_df = pd.read_csv("Sales_April_2019.csv"); # import CSV into df

sales_df = sales_df[["Order ID", "Product"]]; # keeping only Order ID and Product
sales_df = sales_df.reset_index() # reset index

sales_df_pivot = sales_df.pivot_table(index = 'Order ID', columns = 'Product', aggfunc = 'count'); # pivot table

sales_df_pivot[:10]

sales_df_pivot.fillna(0); # replacing all NaN values with 0 (so that df contains only numbers)

sales_df_pivot = sales_df_pivot[sales_df_pivot.columns].astype("bool"); # converts all columns to bools

sales_df_pivot[:10]

sales_df_pivot.value_counts() # calculates support frequency

from mlxtend.frequent_patterns import apriori
sales_apriori = apriori(sales_df_pivot, min_support = 0.00000001,max_len = 2, use_colnames = True);

from mlxtend.frequent_patterns import association_rules
frequent_itemsets = association_rules(frequent_itemsets,metric = "confidence", min_threshold = 0.1)
display(frequent_itemsets)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Kidney Beans),(Eggs),1.0,0.8,0.8,0.80,1.00,0.00,1.0,0.0
1,(Eggs),(Kidney Beans),0.8,1.0,0.8,1.00,1.00,0.00,inf,0.0
2,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf,0.5
3,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6,1.0
4,(Milk),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,0.0
5,(Kidney Beans),(Milk),1.0,0.6,0.6,0.60,1.00,0.00,1.0,0.0
6,(Onion),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,0.0
7,(Kidney Beans),(Onion),1.0,0.6,0.6,0.60,1.00,0.00,1.0,0.0
8,(Kidney Beans),(Yogurt),1.0,0.6,0.6,0.60,1.00,0.00,1.0,0.0
9,(Yogurt),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,0.0
